# overview via code DO NOT RUN

In [1]:
import json
import h5py
import numpy as np
import PIL

import os
current_dir = os.getcwd()
CUR_DIR = current_dir
DATA_HOME_DIR = os.path.join(current_dir,'data')
path = DATA_HOME_DIR + '/' #'/sample/'
test_path = DATA_HOME_DIR + '/test/' #We use all the test data
results_path=DATA_HOME_DIR + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'
batch_size = 64
no_of_epochs = 3

# import theano
# from theano import shared, tensor as T
# from theano.tensor.nnet import conv2d, nnet
# from theano.tensor.signal import pool

from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image

# K.set_image_dim_ordering('th')

# the below runs on python 2 and keras for python 2
# what ive done is laid out the code they use
# then well deconstruct it, then in another notebook
# vgg_yvan, we will do the sam efor my implementation
# the idea is to find what doesnt work about my implementation

# Import our class, and instantiate, lesson1.ipynb
import vgg16; reload(vgg16)
from vgg16 import Vgg16

#instantiate the object
vgg = Vgg16()

# get batches
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)

# finetune/replace final layer
vgg.finetune(batches)

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5105)
/home/yns207/anaconda3/envs/keras-py2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/yns207/anaconda3/envs/keras-py2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/yns207/anaconda3/envs/keras-py2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: D

Found 28336 images belonging to 121 classes.
Found 2000 images belonging to 121 classes.


In [2]:
#fit the nerual net
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
10880/28336 [==========>...................] - ETA: 2974s - loss: 3.7051 - acc: 0.2806

KeyboardInterrupt: 

# import keras and necessary stuff


In [3]:
import json
import numpy as np

# import theano
# from theano import shared, tensor as T
# from theano.tensor.nnet import conv2d, nnet
# from theano.tensor.signal import pool

from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image

# K.set_image_dim_ordering('th')

# setup the neural net

In [4]:
#CHANGES:
#had to add K.set_image_dim_ordering('th') or this code wouldnt run...
# could be point of error

#vgg = Vgg16()
# __init__
FILE_PATH = 'http://files.fast.ai/models/'

#create() helper methods

vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))
def vgg_preprocess(x):
    x = x - vgg_mean
    return x[:, ::-1] # reverse axis rgb->bgr

def ConvBlock(model, layers, filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

def FCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

#create()
model = Sequential()
model.add(Lambda(vgg_preprocess, input_shape=(3,224,224), output_shape=(3,224,224)))

ConvBlock(model, 2, 64)
ConvBlock(model, 2, 128)
ConvBlock(model, 3, 256)
ConvBlock(model, 3, 512)
ConvBlock(model, 3, 512)

model.add(Flatten())
FCBlock(model)
FCBlock(model)
model.add(Dense(1000, activation='softmax'))

fname = 'vgg16.h5'
model.load_weights(get_file(fname, FILE_PATH+fname, cache_subdir='models'))

#get_classes()
fname = 'imagenet_class_index.json'
fpath = get_file(fname, FILE_PATH+fname, cache_subdir='models')
with open(fpath) as f:
    class_dict = json.load(f)

classes = [class_dict[str(i)][1] for i in range(len(class_dict))]

# get batches of data

In [5]:
import os
current_dir = os.getcwd()
CUR_DIR = current_dir
DATA_HOME_DIR = os.path.join(current_dir,'data')
path = DATA_HOME_DIR + '/' #'/sample/'
test_path = DATA_HOME_DIR + '/test/' #We use all the test data
results_path=DATA_HOME_DIR + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'
batch_size = 64
no_of_epochs = 3

# vgg.get_batches()
def get_batches(path, gen=image.ImageDataGenerator(), shuffle=True, batch_size=8, class_mode='categorical'):
        return gen.flow_from_directory(path, target_size=(224,224),
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)
    
batches = get_batches(train_path, batch_size=batch_size)
val_batches = get_batches(valid_path, batch_size=batch_size*2)

Found 28336 images belonging to 121 classes.
Found 2000 images belonging to 121 classes.


# strip the last layer and finetune

In [6]:
model.pop()
for layer in model.layers: layer.trainable=False
model.add(Dense(121, activation='softmax'))
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy', metrics=['accuracy'])
classes = list(iter(batches.class_indices))
for c in batches.class_indices:
    classes[batches.class_indices[c]] = c

In [9]:
#CHANGED nb_sample -> samples
# could be source of error
latest_weights_filename = None 
for epoch in range(no_of_epochs):
    print('running epoch: {}'.format(epoch))
    #batches.samples
    #val_batches.samples
    model.fit_generator(batches, samples_per_epoch=28366, nb_epoch=1,validation_data=val_batches, nb_val_samples=2000)
    latest_weights_filename = 'ft{}.h5'.format(epoch)
    model.save_weights(results_path+latest_weights_filename)
print('completed {} fit operations'.format(no_of_epochs))

running epoch: 0
Epoch 1/1
21312/28366 [=====================>........] - ETA: 1118s - loss: 3.3211 - acc: 0.3273

KeyboardInterrupt: 